<a href="https://colab.research.google.com/github/OneeyedCheshire/COursera-Capstone/blob/master/IBM_Data_Science_Capstone_Week_3_Toronto_Segmentation_Jason_Meil_ipynb_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install packages:
1. beautySoup4 to scrape Wiki page
pip install beautysoup5

2. The xml parser to parse page:
pip install lxmp

pip install html5lib

pip install requests

First import all libraries.

In [0]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

In [2]:

url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


page = requests.get(url)


doc = lh.fromstring(page.content)



tr_elements = doc.xpath('//tr')


[len(T) for T in tr_elements[:12]]


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


In [0]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [0]:
[len(C) for (title,C) in col]

[289, 289, 289]

CREATING  a DataFrame

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.columns=['Borough',"Neighbourhood","Postcode"]
print(df.head())
print(df.shape)

            Borough       Neighbourhood Postcode
0      Not assigned      Not assigned\n      M1A
1      Not assigned      Not assigned\n      M2A
2        North York         Parkwoods\n      M3A
3        North York  Victoria Village\n      M4A
4  Downtown Toronto      Harbourfront\n      M5A
(289, 3)


## Cleaning our data

In [7]:
# 1. removing all occurence of  \n
df=df.replace({'\n': ''}, regex=True)
print(df.head())

            Borough     Neighbourhood Postcode
0      Not assigned      Not assigned      M1A
1      Not assigned      Not assigned      M2A
2        North York         Parkwoods      M3A
3        North York  Victoria Village      M4A
4  Downtown Toronto      Harbourfront      M5A


In [0]:
#2. Dropping  rows with a borough that is Not assigned.
print(df[df['Borough']=="Not assigned"].shape)
df=df[(df['Borough']!="Not assigned")]
df.head()

(77, 3)


,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


We have 77 rows with Borough that is Not assigned.

In [0]:
# Check unique values in each column:
print(len(df.Borough.unique()))
print(len(df.Postcode.unique()))
print(len(df.Neighbourhood.unique()))

11
103
210


We have 11 unique Borough, 103 unique Postcodes, 209 unique Neighbourhood. 


## Processing Data Frame
#4. Combining multiple neighborhood values into one rows for one postal code area.

In [12]:
df_test = df
df_g=df_test.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index()
df_g


,Postcode,Neighbourhood
0,M1A,Not assigned
1,M1B,"Rouge,Malvern"
2,M1C,"Highland Creek,Rouge Hill,Port Union"
3,M1E,"Guildwood,Morningside,West Hill"
4,M1G,Woburn
5,M1H,Cedarbrae
6,M1J,Scarborough Village
7,M1K,"East Birchmount Park,Ionview,Kennedy Park"
8,M1L,"Clairlea,Golden Mile,Oakridge"
9,M1M,"Cliffcrest,Cliffside,Scarborough Village West"


In [13]:
df_part2=df[["Borough","Postcode"]] 
df_part2.head()

,Borough,Postcode
0,Not assigned,M1A
1,Not assigned,M2A
2,North York,M3A
3,North York,M4A
4,Downtown Toronto,M5A


In [0]:
#Joining two dataframes 
df_res=df_g.join(df_part2.set_index('Postcode'), on='Postcode')


In [15]:
#removing duplicates:
df_res_unique = df_res.drop_duplicates(subset=['Postcode', 'Neighbourhood','Borough'], keep=False)
df_res_unique.shape

(122, 3)

In [0]:
df_test = df_res_unique

In [17]:
df_test[df_test['Neighbourhood']=="Not assigned"]

,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
18,M1Y,Not assigned,Not assigned
19,M1Z,Not assigned,Not assigned
20,M2A,Not assigned,Not assigned
21,M2B,Not assigned,Not assigned
22,M2C,Not assigned,Not assigned
23,M2E,Not assigned,Not assigned
24,M2G,Not assigned,Not assigned
33,M2S,Not assigned,Not assigned
34,M2T,Not assigned,Not assigned


In [18]:
#only one value, easely replace it
df_test=df_test.set_value(85, 'Neighbourhood', 'Queen\'s Park')

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/pandas/core/frame.py:2035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[index, col] = value


In [19]:
df_test.head(10)

,Postcode,Neighbourhood,Borough
0,M1A,Not assigned,Not assigned
4,M1G,Woburn,Scarborough
5,M1H,Cedarbrae,Scarborough
6,M1J,Scarborough Village,Scarborough
13,M1S,Agincourt,Scarborough
17,M1X,Upper Rouge,Scarborough
18,M1Y,Not assigned,Not assigned
19,M1Z,Not assigned,Not assigned
20,M2A,Not assigned,Not assigned
21,M2B,Not assigned,Not assigned


In [21]:
df.shape()

TypeError: ignored